<a href="https://colab.research.google.com/github/FatemaTabassum/Data_mining_ML_Revise/blob/main/Advance_data_mining_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For theoretical knowledge you can study following link

https://www.geeksforgeeks.org/machine-learning/naive-bayes-classifiers/

In [ ]:
import re
from collections import defaultdict
import sys
import collections
import math


In [3]:
import re
from collections import defaultdict
import sys
import collections
import math

class NaiveBayesClassifier:
  def __init__(self) -> None:
    self.prior_prob_dict = {}
    self.keep_count_dict = {}
    self.class_1_cnt = 0
    self.class_2_cnt = 0
    self.class_1_identifier = 0
    self.class_2_identifier = 1
    self.dataset_train = []
    self.labels_train = []
    self.dataset_test = []
    self.labels_test = []
    self.max_attribute = 0
    self.attribute_value_counts = defaultdict(set) # To store unique values for each attribute

  #r red from files
  def read_files(self, train_filename, test_filename):
    with open(train_filename, 'r') as f:
      train_set = f.readlines()
    with open(test_filename, 'r') as f:
      test_set = f.readlines()
    return train_set, test_set

  # create a list of dataset
  def create_dataset(self, raw_data_set):
    labels = []
    dataset = []
    max_id = 0
    for line in raw_data_set:
      res_val = re.split(r'[ ]', line.strip()) # Use strip() to remove leading/trailing whitespace
      ind = 0
      attribute_dict = {} # Use a dictionary to store attributes
      for val in res_val:
        if ind == 0:
          if val == '+1':
            labels.append(self.class_1_identifier)      # +1 is categorized as class 0
          else:
            labels.append(self.class_2_identifier)      # -1 is categorized as class 1
        else:
          if ':' in val: # Ensure the value contains ':' before splitting
            v = re.split(r'\:', val)
            id = int(v[0])          # index starts from 1
            num = int(v[1])
            max_id = max(max_id, id)
            attribute_dict[id] = num # Store attribute in dictionary
            self.attribute_value_counts[id].add(num) # Track unique values for smoothing
        # incrementing index for attribute
        ind +=1
      dataset.append(attribute_dict)
    self.max_attribute = max_id # Update the max_attribute instance variable
    return labels, dataset


  def compute_prior_prob(self, labels):
    self.class_1_cnt = labels.count(self.class_1_identifier)
    self.class_2_cnt = labels.count(self.class_2_identifier)
    total_instances = len(labels)
    if total_instances > 0:
        self.prior_prob_dict[self.class_1_identifier] = self.class_1_cnt / total_instances
        self.prior_prob_dict[self.class_2_identifier] = self.class_2_cnt / total_instances
    else:
        # Handle case with no data to avoid division by zero
        self.prior_prob_dict[self.class_1_identifier] = 0
        self.prior_prob_dict[self.class_2_identifier] = 0
    return self.prior_prob_dict



  def classifier_train(self, dataset, labels):
    # Initialize keep_count_dict as a defaultdict to handle missing keys easily
    self.keep_count_dict = defaultdict(lambda: defaultdict(int)) # Nested defaultdict for class -> attribute_id -> value -> count

    for i in range(0, len(dataset)):
        class_label = labels[i]
        attributes = dataset[i]
        for attr_id, value in attributes.items():
            self.keep_count_dict[class_label][attr_id][value] += 1

    #print(self.keep_count_dict) # Optional: for debugging



  def _get_log_conditional_probability(self, class_label, attribute_id, attribute_value):
    """Calculates the log conditional probability of an attribute value given a class."""
    class_count = self.class_1_cnt if class_label == self.class_1_identifier else self.class_2_cnt

    # Get the count of the attribute value for the given class and attribute ID
    count_attribute_value = self.keep_count_dict.get(class_label, {}).get(attribute_id, {}).get(attribute_value, 0)

    # Determine the number of possible values for this attribute based on training data
    num_possible_values = len(self.attribute_value_counts[attribute_id]) if attribute_id in self.attribute_value_counts else 2 # Assume binary if not seen

    # Apply Laplace smoothing (Add-1 smoothing)
    smoothed_count = count_attribute_value + 1
    smoothing_denominator = class_count + num_possible_values

    if smoothing_denominator > 0:
        probability = smoothed_count / smoothing_denominator
        if probability > 0:
            return math.log(probability)
    return 0 # Return log(a very small number) or handle appropriately if probability is zero



  def compute_likelyhood(self, dataset_to_predict):
      predicted_labels = []
      for instance_attributes in dataset_to_predict:
        log_likelihood_class1 = 0
        log_likelihood_class2 = 0

        # Iterate through all unique attribute IDs seen during training or in the current instance
        all_attribute_ids = set(self.attribute_value_counts.keys()) | set(instance_attributes.keys())


        for attribute_id in all_attribute_ids:
          attribute_value = instance_attributes.get(attribute_id, 0) # Get the value if present, default to 0 if not

          # Add log conditional probability for class 1
          log_likelihood_class1 += self._get_log_conditional_probability(self.class_1_identifier, attribute_id, attribute_value)

          # Add log conditional probability for class 2
          log_likelihood_class2 += self._get_log_conditional_probability(self.class_2_identifier, attribute_id, attribute_value)

        # Adding log of prior probabilities
        if self.class_1_identifier in self.prior_prob_dict and self.prior_prob_dict[self.class_1_identifier] > 0:
             log_likelihood_class1 += math.log(self.prior_prob_dict[self.class_1_identifier])

        if self.class_2_identifier in self.prior_prob_dict and self.prior_prob_dict[self.class_2_identifier] > 0:
            log_likelihood_class2 += math.log(self.prior_prob_dict[self.class_2_identifier])


        if log_likelihood_class1 >= log_likelihood_class2:
          predicted_labels.append(self.class_1_identifier)
        else:
          predicted_labels.append(self.class_2_identifier)

      return predicted_labels


  # print helper functions
  def print_lines(self, lines):
    for line in lines:
      print(line)

  def compute_confusion_matrix(self, original_label, computed_labels):
      a = 0
      d = 0
      b = 0
      c = 0
      for i in range(0, len(original_label)):
        if original_label[i] == computed_labels[i] and original_label[i] == 0:
          a += 1
        if original_label[i] == computed_labels[i] and original_label[i] == 1:
          d += 1
        if original_label[i] == 0 and computed_labels[i] == 1:
          b += 1
        if original_label[i] == 1 and computed_labels[i] == 0:
          c += 1
      print('a= ', a, 'b= ', b, 'c= ', c, 'd= ', d)

In [ ]:
if __name__ == "__main__":
    naiveBayes = NaiveBayesClassifier()

    train_set, test_set = naiveBayes.read_file('/home/grads/liza/spring2021/assignment_3/package/breast_cancer.train', '/home/grads/liza/spring2021/assignment_3/package/breast_cancer.test' )
    naiveBayes.print_lines(test_set)
    max_no_attr_train, naiveBayes.labels_train, naiveBayes.dataset_train = naiveBayes.create_dataset(train_set)

    max_no_attr_test, naiveBayes.labels_test , naiveBayes.dataset_test = naiveBayes.create_dataset(test_set)
    max_attr = max(max_no_attr_train, max_no_attr_test)
    naiveBayes.max_attr = max_attr
    naiveBayes.compute_prior_prob(len(naiveBayes.labels_train))
    naiveBayes.classifier_train()

    #test dataset
    computed_labels = naiveBayes.compute_likelyhood(naiveBayes.dataset_test)

    print('#################### Test ########################  ')
    print(naiveBayes.labels_test)
    print(computed_labels)
    naiveBayes.compute_confusion_matrix(naiveBayes.labels_test, computed_labels)


    #train dataset
    computed_labels = naiveBayes.compute_likelyhood(naiveBayes.dataset_train)
    print('#################### train ########################  ')
    print(naiveBayes.labels_train)
    print(computed_labels)
    naiveBayes.compute_confusion_matrix(naiveBayes.labels_train, computed_labels)


    #print(ag.showValues())

# Theory

In the compute_likelyhood method, the log probabilities are added in two main places:

Adding the log of conditional probabilities: Inside the loop that iterates through the attributes (for attr_id in all_attribute_ids:),
the code calculates the log of the conditional probability
for each attribute value given the class and adds it to the class_1_log_likelihood_sum and class_2_log_likelihood_sum.


1. **Here are the relevant lines:**

          if prob_class1 > 0: # Avoid log(0)
            class_1_log_likelihood_sum += math.log(prob_class1)

and

          if prob_class2 > 0: # Avoid log(0)
            class_2_log_likelihood_sum += math.log(prob_class2)

Instead of multiplying probabilities

(P(F1|C) * P(F2|C) * ...), the code calculates the logarithm of each conditional
probability

(log(P(F1|C)), log(P(F2|C)), ...)
and adds them together

(log(P(F1|C)) + log(P(F2|C)) + ...).

This is mathematically equivalent to multiplying the original probabilities because

log(a * b) = log(a) + log(b),

but it avoids potential numerical underflow when dealing with very small probability values.


2. **Adding the log of prior probabilities:**

After the loop through attributes finishes for an instance,
the code adds the log of the prior probability of each class to the accumulated sum of log conditional probabilities.

Here are the relevant lines:
    if self.class_1_identifier in self.prior_prob_dict and self.prior_prob_dict[self.class_1_identifier] > 0:
         class_1_log_likelihood_sum += math.log(self.prior_prob_dict[self.class_1_identifier])


and


    if self.class_2_identifier in self.prior_prob_dict and self.prior_prob_dict[self.class_2_identifier] > 0:
        class_2_log_likelihood_sum += math.log(self.prior_prob_dict[self.class_2_identifier])


This is because, according to Bayes' theorem, the posterior probability is proportional to the likelihood
multiplied by the prior

(P(Class | Features) ∝ P(Features | Class) * P(Class)).

In the log domain, this becomes

log(P(Class | Features)) ∝ log(P(Features | Class)) + log(P(Class)).


The accumulated

class_1_log_likelihood_sum  and  class_2_log_likelihood_sum

represent

log(P(Features | Class)),

so the code adds

log(P(Class))

to them.

By summing the logarithms of the conditional and prior probabilities, the code is effectively calculating the
logarithm of the numerator of Bayes' theorem for each class. The class with the higher resulting sum in the log
domain will also have the higher probability in the original probability domain.

# Reason why we sum the log of the conditional probabilities instead of multiplying the original conditional probabilities

The reason we sum the log of the conditional probabilities instead of multiplying the original conditional probabilities is to avoid numerical underflow.

Here's the explanation:

Naive Bayes Formula: In Naive Bayes, the core calculation for the likelihood of the features given a class is the product of the conditional probabilities of each feature given that class:


P(Features | Class) = P(Feature1 | Class) * P(Feature2 | Class) * ... * P(FeatureN | Class)




Dealing with Many Features: In many real-world datasets, you can have a large number of features (attributes). The conditional probabilities for individual features given a class can be very small numbers (e.g., 0.0001, 0.0000001).

Numerical Underflow: When you multiply many very small floating-point numbers together, the result can become so small that it underflows the precision of the computer's floating-point representation. This means the result might be rounded down to zero, even if the true probability is a very small positive number. If the likelihood for a class becomes zero due to underflow, you can't compare it meaningfully with the likelihoods of other classes, and the classifier won't be able to make a correct prediction.

Using Logarithms: Logarithms have a property that helps with this:



log(a * b * c) = log(a) + log(b) + log(c)




This means that the logarithm of a product is equal to the sum of the logarithms of the individual terms.

Applying to Naive Bayes: By taking the logarithm of the likelihood calculation, the multiplication of many small probabilities is transformed into the summation of their logarithms:


log(P(Features | Class)) = log(P(Feature1 | Class) * ... * P(FeatureN | Class))

log(P(Features | Class)) = log(P(Feature1 | Class)) + ... + log(P(FeatureN | Class))



6. Benefits:

Prevents Underflow: Summing logarithms of small numbers results in a negative number (since log(x) for 0 < x < 1 is negative), but the magnitude of this negative number is much more manageable than the tiny product of the original probabilities. This avoids the underflow issue.
Maintains Relative Order: While the actual numerical values change, the relative order of the likelihoods between different classes is preserved. If P1 > P2, then log(P1) > log(P2). So, comparing log(P(Features | Class1)) and log(P(Features | Class2)) gives the same result as comparing P(Features | Class1) and P(Features | Class2)).
Therefore, summing the log conditional probabilities is a standard and essential technique in implementing Naive Bayes to ensure numerical stability and prevent underflow, allowing the classifier to make accurate predictions even with many features and small probabilities.


# Explaining this line

# all_attribute_ids = set(self.attribute_value_counts.keys()) | set(attributes.keys())



all_attribute_ids = set(self.attribute_value_counts.keys()) | set(attributes.keys())

Here's what it does:

self.attribute_value_counts.keys():

This retrieves all the unique attribute IDs that were encountered during the training process (when create_dataset was called).

attributes.keys():

This retrieves all the attribute IDs that are present in the current test instance being processed in the compute_likelyhood method.

set(...):

Both sets of keys are converted into set objects. Using sets is efficient for performing set operations.

|: This is the set union operator. It combines the two sets of attribute IDs into a single set called all_attribute_ids.

This line is crucial for ensuring that the classifier correctly handles attributes that might be present in the test data but were not present in a specific training instance, or vice-versa.

Why is this important?

In Naive Bayes, to calculate the likelihood of a data point belonging to a class, you need to consider the probabilities of all the relevant features (attributes) given that class.

If an attribute was seen in the training data for a particular class, but is not present in the current test instance, we still need to account for its probability based on the training data.
If an attribute is present in the current test instance but was not seen at all during training
(this is less likely with a well-preprocessed dataset, but possible),
the code still needs to iterate through it to apply smoothing.
By creating all_attribute_ids as the union of attribute IDs from both the training data
(specifically, those seen in attribute_value_counts)

and the current test instance, the code ensures that the loop that calculates the log likelihoods

(for attr_id in all_attribute_ids:)

considers all relevant attributes for that instance, even if an attribute isn't explicitly listed in the sparse representation of that particular test instance. This is essential for applying smoothing correctly and avoiding zero probabilities that could arise from unseen attribute-value combinations.

# Rough - Only for printing

In [ ]:
# nothing related to the algo

# Assuming the NaiveBayesClassifier class is defined in a previous cell
# Let's create a sample raw dataset string
sample_raw_data = """+1 1:1 5:1 10:1
-1 2:1 5:0 8:1
+1 1:0 5:1 12:0"""

# Split the sample data into lines
sample_raw_data_set = sample_raw_data.strip().split('\n')

# Instantiate the NaiveBayesClassifier
nb_classifier = NaiveBayesClassifier()

# Call the create_dataset method with the sample data
max_id, labels, dataset = nb_classifier.create_dataset(sample_raw_data_set)

# Print the results
print(f"max_id: {max_id}")
print(f"labels: {labels}")
print("Sample dataset entries:")
for i in range(min(3, len(dataset))): # Print up to 3 sample entries
  print(f"  {dataset[i]}")

In [ ]:
# This creates a list with 20 zeros
attribute_ar = [0] * 20

# This prints the list
print(attribute_ar)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
